In [1]:
import os
import json
import pandas as pdo
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple
import scanpy as sc
import pickle
import sys

# Ensure the parent directory is in the system path
sys.path.append('/home/minhang/mds_project/sc_classification')
from utils.experiment_manager import ExperimentManager
from utils.experiment_analysis import ExperimentAnalyzer

# surpress warnings
import warnings
warnings.filterwarnings("ignore")

/home/minhang/miniconda3/envs/mds_responder_analysis/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/minhang/miniconda3/envs/mds_responder_analysis/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/minhang/miniconda3/envs/mds_responder_analysis/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/minhang/miniconda3/envs/mds_responder_analysis/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/minhang/miniconda3/envs/mds_

In [2]:
experiment_id = "20250727_234319_fa_100_random_6dbbde08"
experiments_dir = '/home/minhang/mds_project/sc_classification/experiments/'

experiment_manager = ExperimentManager(experiments_dir)
analyzer = ExperimentAnalyzer(experiment_manager)

In [3]:
try:
    experiment = experiment_manager.load_experiment(experiment_id)
    print(f"Loaded experiment: {experiment_id}")
    print(f'DR method: {experiment.config.get("dimension_reduction.method")}')
    print(f'N components: {experiment.config.get("dimension_reduction.n_components")}')
    print(f'Downsampling method: {experiment.config.get("downsampling.method")}')
    print(f'Downsampling rate: {experiment.config.get("downsampling.rate")}')
    print(f'Downsampling seed: {experiment.config.get("downsampling.seed")}')
    print(f'Downsampling random state: {experiment.config.get("downsampling.random_state")}')
    print(f'Downsampling n_jobs: {experiment.config.get("downsampling.n_jobs")}')
except Exception as e:
    print(f"Error loading experiment {experiment_id}: {e}")
    sys.exit(1)

Loading experiment from: /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08
Loaded experiment: 20250727_234319_fa_100_random_6dbbde08
DR method: fa
N components: 100
Downsampling method: random
Downsampling rate: None
Downsampling seed: None
Downsampling random state: None
Downsampling n_jobs: None


#### Generate unsupervised diagnosis plots

In [4]:
analyzer.generate_unsupervised_fa_report(experiment_id=experiment_id)

--- Generating Unsupervised FA Report for Experiment: 20250727_234319_fa_100_random_6dbbde08 ---
Loading experiment from: /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08
Saved SS Loadings plot to: /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08/analysis/unsupervised_dr_analysis/ss_loadings_per_factor.png
Saved Communality plot to: /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08/analysis/unsupervised_dr_analysis/gene_communality_distribution.png


In [5]:
# create a list of factors to compare [X_fa_1, X_fa_2, ..., X_fa_100]
factors_to_compare = [f'X_fa_{i}' for i in range(1, 101)]
print("\n--- EDA: Comparing Factor Loading Distributions ---")
analyzer.plot_factor_loading_distributions(
    experiment_id=experiment_id,
    factors_to_compare=factors_to_compare
)


--- EDA: Comparing Factor Loading Distributions ---
--- Plotting Factor Loading Distributions for 20250727_234319_fa_100_random_6dbbde08 ---
Loading experiment from: /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08
Saved factor loading distribution plot to: /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08/analysis/unsupervised_dr_analysis/factor_loading_distributions.png


#### Generate 2-panels LR-lasso report per-patient 

In [6]:
print("Running LR-Lasso analysis...")
analyzer.generate_lasso_path_2panels_report_internal_CV(experiment_id)  
print("LR-Lasso analysis complete!")

# Check what was created
summary_plots_dir = experiment.get_path('summary_plots')
print(f"\nResults saved to: {summary_plots_dir}")
if summary_plots_dir.exists():
    print("Files created:")
    for file in summary_plots_dir.glob("*.png"):
        print(f"  - {file.name}")

Running LR-Lasso analysis...
--- Generating Internal CV Analysis Report for Experiment: 20250727_234319_fa_100_random_6dbbde08 ---
Loading experiment from: /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08
Loading experiment from: /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08
Generating plot for patient: P01
  Saved plot to /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08/analysis/summary_plots/patient_P01_CV_metrics_and_coefficients.png
Generating plot for patient: P02
  Saved plot to /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08/analysis/summary_plots/patient_P02_CV_metrics_and_coefficients.png
Generating plot for patient: P03
  Saved plot to /home/minhang/mds_project/sc_classification/experiments/20250727_234319_fa_100_random_6dbbde08/analysis/summary_plots/patient_P03_CV_metrics_and_coefficients.